# La Génération de Texte

Pour plus d’information et si vous souhaitez jeter un coup d’oeil au code de huggingface, il est disponible ici : 
https://github.com/huggingface/transformers/tree/main/src/transformers/generation

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# On charge le modèle 
model_name = "gpt2" # Vous pouvez jouer avec différents modèles selon la puissance de votre machine

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "Aussois is" # Texte à tester ici
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(device) # On encode le texte en tokens

output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

Pour changer le modèle utilisé (ici "gpt2"), vous pouvez vous balader sur huggingface, par exemple voir les modèles les plus téléchargés ici :
https://huggingface.co/models?pipeline_tag=text-generation&sort=trending.
Attention cependant à la mémoire. GPT2 tourne très bien sur (presque) n’importe quelle machine, alors que des modèles plus gros seront plus gourmands. Si vous utilisez une machine avec un gpu, pensez à instancier device = "cuda" et y déplacer le modèle ainsi que le texte.

### Beam Search

In [ ]:
output = model.generate(**inputs, num_beams=5, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

Dans toute la suite du TP, on définit pad_token_id=eos.token_id, cela signifie que l’on fait de la génération ouverte. En faisant cela, on indique au modèle que générer du vide est équivalent à terminer la génération.

In [ ]:
# On peut augmenter le nombre de beams, augmenter la longueur, et renvoyer plus qu’une séquence avec num_return_sequences

output = model.generate(**inputs, num_beams=7, max_new_tokens=50, num_return_sequences=3, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 

for generated in output :
    print(tokenizer.decode(generated)) # On convertit les tokens en mots

Le beam search est déterministe, faire tourner les cellules précédentes va donc toujours renvoyer la même chose. On voit bien que les séquences renvoyées sont très proches, on est dans un cas où certains préfixes sont bien meilleurs que les autres aux yeux du modèle.

### Sampling ancestral

In [ ]:
output = model.generate(**inputs, num_beams=1, do_sample=True, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

Le sampling est lui stochastique, et va donc donner des résultats différents à chaque instance. Les résultats peuvent être très surprenants. N’hésitez pas à relancer la cellule suivante jusqu’à avoir des phrases très surprenantes.


In [ ]:
for loop in range(5):
    output = model.generate(**inputs, num_beams=1, do_sample=True, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
    text = tokenizer.decode(output[0]) # On convertit les tokens en mots
    print(f"Génération {loop} : \n{text}")

### Top_k sampling

In [ ]:
output = model.generate(**inputs, num_beams=1, do_sample=True, top_k=40, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

### Nucleus (top_p) sampling

In [ ]:
output = model.generate(**inputs, num_beams=1, do_sample=True, top_p=0.9, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

### Locally typical sampling (typical_p)

In [ ]:
output = model.generate(**inputs, num_beams=1, do_sample=True, typical_p = 0.9, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

### $\eta$-sampling (eta_cutoff)

In [ ]:
output = model.generate(**inputs, num_beams=1, do_sample=True, eta_cutoff=0.003, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

In [ ]:
output = model.generate(**inputs, num_beams=1, do_sample=True, eta_cutoff=0.003, max_new_tokens=50, repetition_penalty=1.2, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

### Playground
On peut cumuler tous ces paramètres afin d’influencer la génération

Comment est-ce que Huggingface arrive à cumuler le Beam Search déterministe avec du sampling stochastique ? Vous pouvez regarder plus en détail ce qui se passe dans le code ici :
https://github.com/huggingface/transformers/blob/main/src/transformers/generation/beam_search.py

In [ ]:
# Ancestral Sampling avec plusieurs beams
output = model.generate(**inputs, num_beams=5, do_sample=True, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

In [ ]:
# On peut regarder les résultats pour les différents beams

output = model.generate(**inputs, num_beams=10, do_sample=True, max_new_tokens=50, num_return_sequences=3, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
for generation in output : 
    print(tokenizer.decode(generation))

In [ ]:
# Top-k avec plusieurs beams
output = model.generate(**inputs, num_beams=5, do_sample=True, top_k=40, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

Quand on cumule les troncations, on applique d’abord top-k puis top-p d’après https://huggingface.co/blog/how-to-generate, mais qu’est-ce qui se passe quand on les cumules toutes ? Vous pouvez jouer avec cela et partager votre génération la plus étonnante avec le reste des participants.

In [ ]:
# Un peu de tout
output = model.generate(**inputs, num_beams=1, do_sample=True, top_k=40, top_p=0.9, eta_cutoff=0.003, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
tokenizer.decode(output[0]) # On convertit les tokens en mots

In [ ]:
# Testez tout
output = model.generate(
    **inputs, 
    num_beams=1, 
    do_sample=True, 
    top_k=40, 
    top_p=0.9, 
    temperature=0.9,
    eta_cutoff=0.003, 
    repetition_penalty=1.2,
    max_new_tokens=50, 
    pad_token_id=tokenizer.eos_token_id) 
tokenizer.decode(output[0]) # On convertit les tokens en mots

## Mesures de diversité de la génération
Ici, on va mesurer à quel point générer plusieurs fois va donner des textes différents, n’hésitez pas à jouer avec tous les paramètres

In [ ]:
import torch
import sacrebleu
import random
import numpy as np

def calculate_average_bleu(generated_texts):
    # Calcul du score BLEU pour chaque paire de textes
    pairwise_bleu_scores = []
    for i in range(len(generated_texts)):
        for j in range(i+1, len(generated_texts)):
            hypothesis = generated_texts[i]
            references = [generated_texts[j]]
            bleu = sacrebleu.corpus_bleu([hypothesis], [references])
            pairwise_bleu_scores.append(bleu.score)
            #print(f"score BLEU entre la génération {i} et {j}: {bleu.score:.2f}")

    # Calcul du BLEU moyen en tant que mesure de diversité
    average_bleu = sum(pairwise_bleu_scores) / len(pairwise_bleu_scores)
    return average_bleu

In [ ]:
prompt = "Aussois is"  # Prompt

# Encodage du prompt
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(device)

# On génère plusieurs textes
num_samples = 5  # Nombre de textes
max_new_tokens = 50  # Nombre de tokens à générer
generated_texts = []

# Boucle de génération
for i in range(num_samples):
    """ Optionnel, on peut ajouter une seed pour reproduire toujours les mêmes 
    torch.manual_seed(i)
    np.random.seed(i)
    random.seed(i)
    """

    # Génération de textes avec différents paramètres
    output = model.generate(
        **inputs,
        num_beams=5,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        #top_k=40,       
        temperature=0.7,
        #top_p=0,
        #eta_cutoff=0,
        #typical_p=0.9,
        #repetition_penalty=1,
        pad_token_id=tokenizer.eos_token_id
    )
    # Décodage des tokens
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_texts.append(generated_text)
    print(f"Sample {i}:\n{generated_text}\n")

avg = calculate_average_bleu(generated_texts)
print(f"Score BLEU moyen :{avg}")

BLEU va mesurer à quel point les phrases sont similaires, plus il est bas, plus les générations sont diverses.

## Beam-search curse
À partir d’une certaine taille, augmenter le nombre de beams va venir raccourcir la génération, voire renvoyer une phrase vide. Ici on fait de la génération sans trop de contraintes mais avec des tâches bien définies c’est plus fragrant.

In [ ]:
# On génère plusieurs textes
num_samples = 5  # Nombre de textes
max_new_tokens = 50  # Nombre de tokens à générer
generated_texts = []

nb_beams = [10, 20, 50, 100]

for j in nb_beams:
    lengths = []
    # Boucle de génération
    for i in range(num_samples):
        """ Optionnel, on peut ajouter une seed pour reproduire toujours les mêmes 
        torch.manual_seed(i)
        np.random.seed(i)
        random.seed(i)
        """

        # Génération de textes avec différents paramètres
        output = model.generate(
            **inputs,
            num_beams=j,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            #top_k=40,       
            temperature=1,
            #top_p=0,
            #eta_cutoff=0,
            #typical_p=0.9,
            #repetition_penalty=1,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )
        # Décodage des tokens
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

        # On calcule la longueur du texte généré (en mots)
        length = len(generated_text.split())
        lengths.append(length)

    avg = calculate_average_bleu(generated_texts)
    # Calcul de la longueur moyenne pour ce nombre de beams
    avg_length = sum(lengths) / len(lengths)
    print(f"Score Bleu moyen pour {j} beams : {avg:.5f}, Longueur moyenne : {avg_length:.2f} mots\n")
    

In [ ]:
print(generated_texts)

La longueur moyenne des textes a tendance a diminuer quand le nombre de beams devient très grand

## Conditions d’arrêt du beam search

In [ ]:
# On génère plusieurs textes
num_samples = 5  # Nombre de textes
max_new_tokens = 50  # Nombre de tokens à générer
generated_texts = []

# Différentes valeurs de early_stopping
early_stopping_options = [False, True]

# On fixe un nombre de beams
num_beams = 20

for es in early_stopping_options:
    lengths = []
    # Boucle de génération
    for i in range(num_samples):
        """ Optionnel, on peut ajouter une seed pour reproduire toujours les mêmes 
        torch.manual_seed(i)
        np.random.seed(i)
        random.seed(i)
        """

        # Génération de textes avec différents paramètres (vous pouvez les changer à votre guise)
        # Ici, on ne varie que early_stopping 
        output = model.generate(
            **inputs,
            num_beams=num_beams,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            early_stopping=es,
            pad_token_id=tokenizer.eos_token_id
        )
        # Décodage des tokens
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

        # On calcule la longueur du texte généré (en mots, approx. via split)
        length = len(generated_text.split())
        lengths.append(length)

    avg = calculate_average_bleu(generated_texts)
    # Calcul de la longueur moyenne pour cette configuration
    avg_length = sum(lengths) / len(lengths)
    print(f"Pour early_stopping={es}, Score BLEU moyen : {avg:.5f}, Longueur moyenne : {avg_length:.2f} mots\n")


In [ ]:
# Ici on va faire varier la length_penalty pour regarder son effet

# On génère plusieurs textes
num_samples = 5  # Nombre de textes
max_new_tokens = 50  # Nombre de tokens à générer
generated_texts = []

# On fixe un nombre de beams et on fait varier la length_penalty
num_beams = 20
length_penalties = [0.8, 1.0, 1.2, 1.5, 2, 3]

for lp in length_penalties:
    lengths = []
    # Boucle de génération
    for i in range(num_samples):
        """ Optionnel, on peut ajouter une seed pour reproduire toujours les mêmes 
        torch.manual_seed(i)
        np.random.seed(i)
        random.seed(i)
        """

        # Génération de textes avec les paramètres fixes et la length_penalty variable
        output = model.generate(
            **inputs,
            num_beams=num_beams,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=1,
            early_stopping=True,       # On garde l'arrêt anticipé
            length_penalty=lp,         # On fait varier cette pénalité
            pad_token_id=tokenizer.eos_token_id
        )
        # Décodage des tokens
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

        # On calcule la longueur du texte généré (en mots, approx. via split)
        length = len(generated_text.split())
        lengths.append(length)

    avg = calculate_average_bleu(generated_texts)
    avg_length = sum(lengths) / len(lengths)
    print(f"Score BLEU moyen pour length_penalty={lp} : {avg:.5f}, Longueur moyenne (en mots) : {avg_length:.2f}\n")

# Calculs de perplexité

La perplexité est une mesure de surprise du texte, plus elle est élevée moins le modèle associe de fortes probabilités aux mots de la séquence.

In [ ]:
import torch
from tqdm import tqdm
import torch.nn.functional as F

# Inspiré par l’implémentation HuggingFace disponible ici : https://huggingface.co/docs/transformers/perplexity

# Méthode 1, avec tout le contexte à chaque fois 
def compute_token_by_token_ppl(model, encodings):
    input_ids = encodings.input_ids.to(device)
    seq_len = input_ids.size(1)
    if seq_len > model.config.n_positions:
        return "Ce texte est trop long pour la fenêtre de contexte du modèle"
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    
    logits = outputs.logits  # shape: [batch_size, seq_length, vocab_size]
    
    # Shift pour prédire le token i à la position i-1
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = input_ids[:, 1:].contiguous()

    log_probs = F.log_softmax(shift_logits, dim=-1)
    # On gather pour regarder les probabilités du token qui est renvoyé
    token_log_probs = log_probs.gather(-1, shift_labels.unsqueeze(-1)).squeeze(-1)
    
    average_nll = -token_log_probs.mean() 
    token_by_token_ppl = torch.exp(average_nll)
    return token_by_token_ppl.item()

In [ ]:
# On peut aller plus loin encore et afficher la probabliité associée à chaque token d’une phrase en modifiant cette fonction :

def compute_token_probs_and_ppl(model, tokenizer, encodings, device=device):
    input_ids = encodings.input_ids.to(device)
    seq_len = input_ids.size(1)

    if seq_len > model.config.n_positions:
        return "Ce texte est trop long pour la fenêtre de contexte du modèle"
    
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)

    logits = outputs.logits  # [batch_size, seq_length, vocab_size]

    # On shift pour prédire le token t+1 à la position t
    shift_logits = logits[:, :-1, :].contiguous()    # Prédictions 
    shift_labels = input_ids[:, 1:].contiguous()     # tokens prédits

    # Log probs de chaque token
    log_probs = F.log_softmax(shift_logits, dim=-1)
    # Log probs de chaque token prédit
    token_log_probs = log_probs.gather(-1, shift_labels.unsqueeze(-1)).squeeze(-1)  # [batch_size, seq_length-1]

    average_nll = -token_log_probs.mean()
    token_by_token_ppl = torch.exp(average_nll)

    # On passe des log probs aux probabilités
    token_probs = torch.exp(token_log_probs)  # shape: [batch_size, seq_length-1]
    token_probs = token_probs.squeeze(0)      # batch_size=1

    # On commence à partir du second token de la phrase, le premier n’ayant pas de contexte autre que <s>
    predicted_tokens = shift_labels.squeeze(0)  # shape: [seq_length-1]

    print("Token-by-token probabilities:")
    for i, token_id in enumerate(predicted_tokens):
        token_str = tokenizer.decode([token_id])
        prob = token_probs[i].item()
        print(f"Token: {token_str} | Probability: {prob:.6f}")

    return token_by_token_ppl.item()

In [ ]:
prompt_test = "The Eiffel Tower is located in"
inputs = tokenizer(prompt_test, return_tensors="pt", add_special_tokens=False).to(device)

output = model.generate(**inputs, num_beams=5, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id) # On génère des tokens 
sortie = tokenizer.decode(output[0]) # On convertit les tokens en mots
print(sortie)

Regardons maintenant à quel point GPT2 est sûr de lui à chaque étape, n’hésitez pas à changer le prompt de test

In [ ]:
inputs = tokenizer(sortie, return_tensors="pt", add_special_tokens=False).to(device)

compute_token_probs_and_ppl(model, tokenizer, inputs)

Maintenant, regardons comment faire dans le cas où le contexte est trop grand pour le modèle, le contexte de GPT2 est de 1024 par exemple (j’ai demandé un texte super long à ChatGPT, vous pouvez également proposer le vôtre).

In [ ]:
text = "On a crisp autumn morning, Eleanor stepped outside her small cottage at the edge of the forest and began walking down a narrow path lined with amber and russet leaves. The world had changed colors overnight, as if an unseen painter had swept across the landscape with a palette of warm hues. She breathed in the scent of damp earth, distant pine, and the faint aroma of woodsmoke drifting from a neighbor’s chimney. Although the air carried a slight chill, the day promised gentle sunshine and light winds, perfect for a long and thoughtful stroll. As she continued along the winding trail, Eleanor recalled the stories her grandfather had told her when she was a child—tales of hidden groves and forgotten clearings deep in the woods, where ancient trees whispered secrets to one another. He had insisted that if one listened carefully enough, the wind through the branches carried voices from centuries past. She had never been entirely sure whether to believe him, but as she walked, she allowed herself the luxury of imagining these old legends might hold some truth. There was comfort in thinking that one’s ancestors might leave behind faint echoes, lingering in the quiet corners of nature. About half a mile in, Eleanor reached a small stream. Its waters, clear and cold, danced over smooth stones, creating a soft, melodic murmur. She paused to watch leaves float downstream, each one a tiny vessel drifting toward unknown destinations. The sunlight, filtering through the half-bare branches, reflected in bright spots off the ripples, reminding her that beauty often lay in small, transient details. She took her time before moving on, feeling as if each moment was a gift she shouldn’t rush. Not long after crossing the stream via a makeshift wooden plank, she came across a clearing she did not recognize. It was shaped like an oval and ringed with young birch trees whose bark gleamed pale against the darker backdrop of firs and oaks. In the center stood a solitary stone bench, weathered and covered in moss, as though it had been placed there by someone who valued solitude. Eleanor brushed off some of the moss and sat, resting her legs and taking in the silent theater around her. No birds chirped, no squirrels chattered. It was as if this spot had declared itself a sanctuary from noise. While resting, she thought about the countless times she had ventured outdoors in search of nothing in particular: just the quiet companionship of trees, the patient passage of clouds, and the sound of her own footsteps on the trail. She considered how, in these quiet moments, she often found a clarity that eluded her in the hustle of daily life. Back at her cottage, there were chores to be done, letters to answer, and errands waiting. Out here, these demands receded, leaving room for the warmth of memories and the subtle interplay of time and stillness. Refreshed by the pause, Eleanor stood and continued forward, leaving the clearing behind. Eventually, the path widened, and she found herself walking alongside an old stone wall, partially collapsed in places. Vines and moss had claimed it, weaving new textures and patterns that hinted at the slow, persistent artistry of nature over generations. She liked to imagine who might have built the wall—farmers long ago clearing land, or perhaps villagers marking a boundary. The wall, now broken and quiet, bore silent witness to a past she could only guess at. As noon approached, the sunlight grew warmer, and the forest seemed to awaken. A distant woodpecker tapped at a trunk, small birds fluttered between branches, and a gentle breeze carried the distant laughter of someone working in a nearby orchard. Eleanor knew there was a village not far beyond the forest’s border, where life continued its pleasant rhythm: apples harvested, bread baked, stories swapped over steaming cups of tea. Soon, she would turn back toward her cottage, but not just yet. The day still had hours left to unfold. After another half-hour of walking, she reached a hillside that offered a view of rolling fields beyond the trees. Patches of farmland, dotted with hay bales, stretched toward a line of distant hills. A single hawk circled overhead, its keen eyes scanning the ground below for a quick meal. Eleanor watched the hawk’s flight, feeling a quiet admiration for its independence and grace. When she finally turned around to retrace her steps home, she found the forest just as welcoming as before. The path felt familiar but not stale; rather, it was like greeting an old friend. She noticed details she had missed earlier—a cluster of mushrooms at the base of a beech tree, the gentle slope of the trail as it curved around a thicket. Each step brought her closer to the cottage, and with it the ordinary tasks of her life, but she carried within her a renewed sense of peace. By the time Eleanor stepped through her front door, the golden afternoon light had begun to angle across the floorboards. She placed a kettle on the stove, choosing a fragrant herbal blend for her tea. Waiting for the water to boil, she looked out the window at the edge of the forest, grateful that she had taken the time to wander among the trees. There was a quiet magic in that forest—subtle, unassuming, yet undeniably present. It asked for nothing and offered everything: a calm place to think, to listen, to remember. And tomorrow, or perhaps the day after, she might return to discover something new, or to simply be, wrapped in the gentle hush of autumn leaves and whispered histories."

inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False).to(device)

In [ ]:
def compute_context_window_ppl(model, encodings, window_size):
    input_ids = encodings.input_ids.to(device)
    seq_len = input_ids.size(1)
    max_length = window_size

    window_losses = []
    for start_idx in range(0, seq_len, max_length):
        end_idx = min(start_idx + max_length, seq_len)
        window_input_ids = input_ids[:, start_idx:end_idx]

        with torch.no_grad():
            outputs = model(window_input_ids, labels=window_input_ids)
        
        # outputs.loss représente la NLL moyenne sur tous les tokens dans la fenêtre
        window_loss = outputs.loss
        window_losses.append(window_loss)

    average_loss = torch.stack(window_losses).mean()
    context_window_ppl = torch.exp(average_loss)
    return context_window_ppl.item()

In [ ]:
def compute_perplexity_with_half_window_context(model, encodings, window_size):
    half_window = window_size // 2

    input_ids = encodings.input_ids.to(device)
    seq_len = input_ids.size(1)

    nlls = []
    start_positions = range(0, seq_len, half_window)
    for start_pos in tqdm(start_positions):
        end_pos = start_pos + window_size
        # Si on a plus assez de tokens, on break ou on peut calculer la ppl des tokens restants en mettant end_pos = seq_len
        if end_pos > seq_len:
            end_pos = seq_len
            break

        # On choisit notre fenêtre
        window_input_ids = input_ids[:, start_pos:end_pos]

        # Seule la deuxième moitié de la fenêtre de contexte sert à scorer notre texte
        target_ids = window_input_ids.clone()
        # On masque la première moitié du texte (-100 est une valeur arbitraire)
        target_ids[:, :half_window] = -100

        with torch.no_grad():
            outputs = model(window_input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
            nlls.append(neg_log_likelihood)
        
        if end_pos == seq_len:
            break

    # On récupère la perplexité moyenne
    average_nll = torch.stack(nlls).mean()
    ppl = torch.exp(average_nll)
    return ppl.item()

In [ ]:
compute_token_by_token_ppl(model, inputs)

In [ ]:
compute_perplexity_with_half_window_context(model, inputs, window_size=512)

In [ ]:
compute_context_window_ppl(model, inputs, window_size=512)

On obtient différents résultats de perplexité à cause des différentes façons de calculer. Sur les texte très longs, ces différences peuvent devenir très grandes.

# Minimum Bayes Risk Decoding

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import sacrebleu

device = "cuda" if torch.cuda.is_available() else "cpu"

# On charge le modèle NLLB
model_name = "facebook/nllb-200-distilled-600M"
source_lang = "eng_Latn"  # Langue source (Anglais)
target_lang = "fra_Latn"  # Langue cible (Français)

tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=source_lang, tgt_lang=target_lang)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Pour NLLB, il faut spécifier la langue cible en préfixant le token <2lang_code>
# Ici, on force l'utilisation du bos_token correspondant à la langue cible
forced_bos_token_id = tokenizer.convert_tokens_to_ids("fra_Latn")

# Texte source à traduire
prompt = "Aussois is a wonderful place"

# Encodage du prompt
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(device)

# Génération des références (candidats "best") de manière déterministe
nb_best = 5
best_set = []
for i in range(nb_best):
    output = model.generate(
        **inputs,
        max_new_tokens=10,
        num_beams=5,  # On utilise le beam search pour une génération plus déterministe
        forced_bos_token_id=forced_bos_token_id
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    best_set.append(generated_text)

# Génération de candidats divers (monte_carlo_set) de manière stochastique
nb_divers = 30
monte_carlo_set = []
for i in range(nb_divers):
    output = model.generate(
        **inputs,
        do_sample=True,
        max_new_tokens=10,
        top_k=40,  # Pour plus de diversité
        forced_bos_token_id=forced_bos_token_id
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    monte_carlo_set.append(generated_text)

In [ ]:
def select_best_candidate(best_set, monte_carlo_set): 
    # Ici on cherche le candidat qui ressemble le plus à tous les autres selon le BLEU moyen
    scores = []
    for candidate in best_set:
        pairwise_bleu_scores = []
        for reference_text in monte_carlo_set:
            bleu = sacrebleu.corpus_bleu([candidate], [[reference_text]])
            pairwise_bleu_scores.append(bleu.score)

        # Calcul du BLEU moyen pour ce candidat
        average_bleu = sum(pairwise_bleu_scores) / len(pairwise_bleu_scores)
        scores.append(average_bleu)

    # On sélectionne le candidat avec le score moyen le plus élevé
    best_index = max(range(len(scores)), key=lambda idx: scores[idx])
    return best_set[best_index], scores[best_index]


In [ ]:
candidat, score = select_best_candidate(best_set, monte_carlo_set)
print(f"La meilleure traduction est :\n{candidat}\navec un BLEU moyen de {score:.5f}")

# Monte Carlo Tree Search

In [ ]:
from transformers import AutoModelForSequenceClassification

# On charge le modèle chargé de scorer nos textes
# C’est un modèle très simple chargé de donner un score de polarité positif et négatif aux textes, chacun entre 0 et 1
scoring_model_name = "siebert/sentiment-roberta-large-english"
scoring_tokenizer = AutoTokenizer.from_pretrained(scoring_model_name)
scoring_model = AutoModelForSequenceClassification.from_pretrained(scoring_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

scoring_model.to(device)
scoring_model.eval()

def get_positive_score(texts):
    """
    On calcule le sentiment (positif)
    """
    inputs = scoring_tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        outputs = scoring_model(**inputs)
    
    # Apply softmax to get probabilities
    probabilities = torch.softmax(outputs.logits, dim=1)
    
    positive_scores = probabilities[:, 1]  # Index 1 représente le score "positif"

    # Il faut choisir si l’on veut un texte positif ou négatif en commentant la ligne inutile
    scores = positive_scores               # On veut que le texte généré soit plutôt positif
    
    return scores

def get_negative_score(texts):
    """
    On calcule le sentiment (négatif)
    """
    inputs = scoring_tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        outputs = scoring_model(**inputs)
    
    # Apply softmax to get probabilities
    probabilities = torch.softmax(outputs.logits, dim=1)
    
    negative_scores = probabilities[:, 0]  # Index 0 représente le score "négatif"

    # Il faut choisir si l’on veut un texte positif ou négatif en commentant la ligne inutile
    scores = negative_scores               # On veut que le texte généré soit plutôt négatif
    
    return scores

# Exemple
text = "This movie was terrible"
positive_score = get_positive_score([text])
negative_score = get_negative_score([text])
print(f"Positive score: {positive_score[0].item():.4f}") 
print(f"Negative score: {negative_score[0].item():.4f}") 

In [ ]:
from mcts_script import main

# Arguments
c = 1.0              # Constante d’exploration, plus haut = plus d’exploration des noeuds les moins visités
alpha = 1.0          # Priorité donnée aux probas du modèle ou au score que l’on renvoie
temperature = 0.7    # Température utilisée pendant la génération
penalty = 1.1        # Pénalité de répétition
num_it = 50          # Nombre d’itérations MCTS par token
prompt_text = "This movie was"


# On va générer un texte "positif"
main(c, alpha, temperature, penalty, num_it, get_positive_score, prompt_text)

In [ ]:
# Même chose avec du texte que l’on veut "négatif"

main(c, alpha, temperature, penalty, num_it, get_negative_score, prompt_text)